# Quantitative metrics for image-patches

In [ ]:
import os
import sys
import csv

from scipy import ndimage as nd
from skimage import measure



current_path = os.path.abspath('.')
root_path = os.path.dirname(os.path.dirname(current_path))
sys.path.append(root_path)

from sourcecode.BONE_CHANNELS.bones_dataloader import *
from sourcecode.wsi_image_utils import *
from sourcecode.evaluation_utils import *

number_epoches = 100
dataset_name = "BONE_CHANNELS"
dataset_dir = "../../datasets/" + dataset_name
batch_size = 1
patch_size = 640
patch_size_tuple = (patch_size, patch_size)
color_model = "LAB"
class_name = "bones"
dataloaders = create_dataloader(tile_size="{}x{}".format(patch_size, patch_size),
                                batch_size=batch_size, 
                                shuffle=False,
                                img_input_size=patch_size_tuple,
                                img_output_size=patch_size_tuple,
                                dataset_dir=dataset_dir,
                                color_model=color_model)

dataset_train_size = len(dataloaders['train'].dataset)
dataset_test_size = len(dataloaders['test'].dataset)
print("-")

tile_size = 20
magnification=0.625

threshold_itc = 200/(0.243 * pow(2, 5))

#wsi_images_dir_normal = "{}/testing/normal/wsi".format(dataset_dir)
wsi_images_dir_tumor = "{}/testing/bones".format(dataset_dir)

trained_model_version = f"{dataset_name}__Size-{patch_size}x{patch_size}_Epoch-{number_epoches}_Images-{dataset_test_size}_Batch-1__random_distortion"
results_dir="{}/results/{}/testing".format(dataset_dir, trained_model_version)

# Arquivo com as métricas médias para cada p
csv_mean_file_path = "{}/pixels/mean_quantitative_analysis.csv".format(results_dir)
mean_file = open(csv_mean_file_path, newline='', mode='w')
mean_writer = csv.writer(mean_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
mean_writer.writerow(['wsi_image', 'class', 'mean auc', 'mean accuracy', 'mean precision', 'mean f1/dice', 'mean jaccard', 'mean sensitivity/recall', 'mean specificity', 'mean tp', 'mean tn', 'mean fp', 'mean fn'])

wsi_tissue_patches = {}

In [ ]:
import numpy as np
from PIL import Image
from scipy import ndimage
from skimage.measure import regionprops
import os
import matplotlib.pyplot as plt

def fill_contours(arr):
    return np.maximum.accumulate(arr,1) & np.maximum.accumulate(arr[:,::-1],1)[:,::-1]

def get_mean_props_area(RGBna):
    labeled, nr_objects = ndimage.label(RGBna != 0.) 
    props = regionprops(labeled)

    if nr_objects == 0:
        return 0
    
    mean_props_area = 0
    for prop in range(len(props)):
        mean_props_area = mean_props_area + props[prop].area
    
    mean_props_area = mean_props_area/len(props)
    return mean_props_area

# Remove ruídos  
def remove_noise(RGBna, mean_props_area):
    labeled, _ = ndimage.label(RGBna != 0.) 
    props = regionprops(labeled)

    new_mask = np.copy(RGBna)
    
    for prop in range(len(props)):
        if props[prop].area > (0.1)*mean_props_area:
            continue
        else:
            [X, Y, x, y] = props[prop].bbox
            new_mask[X:x, Y:y] = props[prop].image_filled.astype(np.uint8)*0
       
            
    #plt.imshow(new_mask)#Image.fromarray(new_mask).show()
    return new_mask

# Preenche contornos  
def fill_components_contours(RGBna):
    labeled, nr_objects = ndimage.label(RGBna != 0.) 
    props = regionprops(labeled)

    new_mask = np.copy(RGBna)

    for prop in range(len(props)):
        [X, Y, x, y] = props[prop].bbox
        new_mask[X:x, Y:y] = fill_contours(props[prop].image_filled.astype(np.uint8)*255)

    return new_mask

## Evaluation varying p

In [ ]:
for threshold_prob in np.arange(0.0,1.0,0.05):
    csv_file_path = f'{results_dir}/pixels/quantitative_analysis_{"%.3f" % threshold_prob}.csv'

    wsi_tissue_patches = {}
    with open(csv_file_path, newline='', mode='w') as medidas_file:
        medidas_writer = csv.writer(medidas_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        medidas_writer.writerow(['wsi_image', 'patch_image', 'auc', 'accuracy', 'precision', 'f1/dice', 'jaccard', 'sensitivity/recall', 'specificity', 'pixels', 'tp', 'tn', 'fp', 'fn'])

        for batch_idx, (data, target, fname, original_size) in enumerate(dataloaders['test']):

            sum_auc = 0.0 
            sum_accuracy = 0.0 
            sum_precision = 0.0 
            sum_f1 = 0.0 
            sum_jaccard = 0.0
            sum_recall = 0.0
            sum_specificity = 0.0
            sum_tp = 0.0
            sum_tn = 0.0
            sum_fp = 0.0
            sum_fn = 0.0

            # wsi image number
            wsi_image_number = fname[0].split("_")[0]
            if wsi_image_number not in wsi_tissue_patches:

                # extract the tissue region from original image and draw the heat grid
                wsi_image_path = "{}/{}.png".format(wsi_images_dir_tumor, wsi_image_number)
                #if not os.path.exists(wsi_image_path):
                #    wsi_image_path = "{}/{}.tif".format(wsi_images_dir_normal, wsi_image_number)

                 # scale down image
                print(wsi_image_path)
                wsi_image = open_wsi(wsi_image_path)
                pil_scaled_down_image = scale_down_wsi(wsi_image, magnification, False)
                np_scaled_down_image = pil_to_np(pil_scaled_down_image)

                # extract tissue region 
                np_tissue_mask, np_masked_image = extract_normal_region_from_wsi(wsi_image_path, np_scaled_down_image, None)
                pil_masked_image = np_to_pil(np_masked_image)

                # draw the heat grid
                pil_img_result, heat_grid, number_of_tiles = draw_heat_grid(np_masked_image, tile_size)

                tissue_patches = []
                for idx, (position, row, column, location, size, color) in enumerate(heat_grid):
                    if color != GREEN_COLOR: 
                        tissue_patches.append("{}_r{}c{}.png".format(wsi_image_number, row, column))

                wsi_tissue_patches[wsi_image_number] = tissue_patches
                #print(wsi_tissue_patches)

            # check if the patch was excluded in preprocessing step
            patch_excludde_in_preprocessing = fname[0] not in wsi_tissue_patches[wsi_image_number]

            # load the mask image
            mask_np_img = target[0].numpy()

            # roi x non_roi classes
            wsi_class = class_name if wsi_image_path.find(class_name) > 0 else "normal"
            patch_class = "roi" if np.max(np.unique(mask_np_img)) > 0 else 'non_roi'


            # load the predicted image result
            patch_results_dir = "{}/{}/patch/{}x{}/{}".format(results_dir, wsi_class, patch_size, patch_size, fname[0])
            print("Patch results dir: " + patch_results_dir)
            unet_result_img = "{}/01-unet_result/{}".format(patch_results_dir, fname[0])
            predicted_pil_img = Image.fromarray(np.zeros(mask_np_img.shape)) if patch_excludde_in_preprocessing else load_pil_image(unet_result_img, gray=True) if os.path.isfile(unet_result_img) else Image.fromarray(np.zeros(mask_np_img.shape))
            predicted_np_img = np.copy(pil_to_np(predicted_pil_img)) 
            predicted_np_img = predicted_np_img * (1.0/255)
            predicted_np_img = basic_threshold(predicted_np_img, threshold=threshold_prob, output_type="float")

            predicted_labels = measure.label(predicted_np_img, connectivity=2)
            #predicted_np_img = np.zeros((predicted_np_img.shape[0], predicted_np_img.shape[1]))
            #labels = np.unique(predicted_labels)
            #properties = measure.regionprops(predicted_labels)
            #for lbl in range(1, np.max(labels)):
            #    major_axis_length = properties[lbl-1].major_axis_length
            #    if major_axis_length > threshold_itc:
            #        predicted_np_img[predicted_labels == lbl] = 1

            # SAVE BINARY IMAGES
            bin_images_path = f'../../datasets/BONE_CHANNELS/results/BONE_CHANNELS__Size-640x640_Epoch-100_Images-464_Batch-1__random_distortion/testing/bones/patch/640x640/binary_images/prob_{"%.3f" % threshold_prob}/'
            if not os.path.isdir(bin_images_path):
                os.mkdir(bin_images_path)

            predicted_np_img_255 = predicted_np_img * 255
            mean = get_mean_props_area(predicted_np_img_255)
            predicted_np_img_255 = remove_noise(predicted_np_img_255, mean)
            predicted_np_img_255 = fill_components_contours(predicted_np_img_255)
    
            new_p = Image.fromarray(predicted_np_img_255)
            if new_p.mode != 'RGB':
                new_p = new_p.convert('RGB')
            new_p.save(bin_images_path + fname[0])

            # metrics
            auc = roc_auc_score(mask_np_img, predicted_np_img)
            precision = precision_score(mask_np_img, predicted_np_img)
            recall = recall_score(mask_np_img, predicted_np_img)
            accuracy = accuracy_score(mask_np_img, predicted_np_img)
            f1 = f1_score(mask_np_img, predicted_np_img)
            specificity = specificity_score(mask_np_img, predicted_np_img)
            jaccard = jaccard_score(mask_np_img, predicted_np_img)

            total_pixels, tn, fp, fn, tp = tn_fp_fn_tp(mask_np_img, predicted_np_img)

            print("Results for {:26} ({:7} - {:8} - {:04.2f} accuracy)".format(fname[0].split("_")[1], patch_class, "excluded" if patch_excludde_in_preprocessing else "unet", accuracy))
            print("   Precision: \t{}".format(precision))
            print("   Recall/Sen: \t{}".format(recall))
            print("   F1/Dice: \t{}".format(f1))
            print("   Accuracy: \t{}".format(accuracy))
            print("   Specificity: {}".format(specificity))
            print("   Jaccard: \t{}".format(jaccard))
            print("   TP = {} TN = {} FP = {} FN = {}".format(tp, tn, fp, fn))
            print("-")

            medidas_writer.writerow([wsi_image_number, patch_class, auc, accuracy, precision, f1, jaccard, recall, specificity, total_pixels, tp, tn, fp, fn])

            sum_auc = sum_auc + auc 
            sum_accuracy = sum_accuracy + accuracy
            sum_precision = sum_precision + precision 
            sum_f1 = sum_f1 + f1
            sum_jaccard = sum_jaccard + jaccard
            sum_recall = sum_recall + recall
            sum_specificity = sum_specificity + specificity
            sum_tp = sum_tp + tp
            sum_tn = sum_tn + tn
            sum_fp = sum_fp + fp
            sum_fn = sum_fn + fn
            
    mean_writer.writerow([wsi_image_number, 
                          patch_class, 
                          sum_auc/dataset_test_size, 
                          sum_accuracy/dataset_test_size, 
                          sum_precision/dataset_test_size, 
                          sum_f1/dataset_test_size, 
                          sum_jaccard/dataset_test_size,
                          sum_recall/dataset_test_size,
                          sum_specificity/dataset_test_size,
                          sum_tp/dataset_test_size,
                          sum_tn/dataset_test_size,
                          sum_fp/dataset_test_size,
                          sum_fn/dataset_test_size])


In [6]:
import numpy as np
import pandas as pd
import os

dir = f'../../datasets/BONE_CHANNELS/results/BONE_CHANNELS__Size-640x640_Epoch-100_Images-464_Batch-1__random_distortion/testing/pixels/'
files = os.listdir(dir)

means_matrix = np.zeros((20, 12))
for idx in range(len(files)):
    if os.path.isdir(dir + files[idx]):
        continue
    
    print(files[idx])
    my_data = np.genfromtxt(dir + files[idx], delimiter=',')

    my_data = np.delete(my_data, 0, 0)
    my_data = np.delete(my_data, 0, 1)
    my_data = np.delete(my_data, 0, 1)

    means_row = []
    for col in range(len(my_data[0,:])):
        means_row.append(np.sum(my_data[:,col])/(len(my_data[:,col])))

    means_matrix[idx,:] = np.array(means_row)

np.round(means_matrix, 3, means_matrix)
df = pd.DataFrame(means_matrix)
df['file'] = files
# Save DataFrame to .csv
df.to_csv(dir + 'means.csv', index=False, header=['auc', 'accuracy', 'precision', 'f1/dice', 'jaccard', 'sensitivity/recall', 'specificity', 'pixels', 'tp', 'tn', 'fp', 'fn', 'file'])
#print(means_matrix)

quantitative_analysis_0.000.csv
quantitative_analysis_0.050.csv
quantitative_analysis_0.100.csv
quantitative_analysis_0.150.csv
quantitative_analysis_0.200.csv
quantitative_analysis_0.250.csv
quantitative_analysis_0.300.csv
quantitative_analysis_0.350.csv
quantitative_analysis_0.400.csv
quantitative_analysis_0.450.csv
quantitative_analysis_0.500.csv
quantitative_analysis_0.550.csv
quantitative_analysis_0.600.csv
quantitative_analysis_0.650.csv
quantitative_analysis_0.700.csv
quantitative_analysis_0.750.csv
quantitative_analysis_0.800.csv
quantitative_analysis_0.850.csv
quantitative_analysis_0.900.csv
quantitative_analysis_0.950.csv


## Generate masks for best p (0.4)

In [ ]:
import numpy as np
from PIL import Image
from scipy import ndimage
from skimage.measure import regionprops
import os
import matplotlib.pyplot as plt

def fill_contours(arr):
    return np.maximum.accumulate(arr,1) & np.maximum.accumulate(arr[:,::-1],1)[:,::-1]

def get_mean_props_area(RGBna):
    labeled, nr_objects = ndimage.label(RGBna != 0.) 
    props = regionprops(labeled)

    new_mask = np.copy(RGBna)
    mean_props_area = 0
    for prop in range(len(props)):
        mean_props_area = mean_props_area + props[prop].area
    
    mean_props_area = mean_props_area/len(props)
    return mean_props_area

# Remove ruídos  
def remove_noise(RGBna, mean_props_area):
    labeled, _ = ndimage.label(RGBna != 0.) 
    props = regionprops(labeled)

    new_mask = np.copy(RGBna)
    
    for prop in range(len(props)):
        if props[prop].area > (0.1)*mean_props_area:
            continue
        else:
            [X, Y, x, y] = props[prop].bbox
            new_mask[X:x, Y:y] = props[prop].image_filled.astype(np.uint8)*0
       
            
    #plt.imshow(new_mask)#Image.fromarray(new_mask).show()
    return new_mask

# Preenche contornos  
def fill_components_contours(RGBna):
    labeled, nr_objects = ndimage.label(RGBna != 0.) 
    props = regionprops(labeled)

    new_mask = np.copy(RGBna)

    for prop in range(len(props)):
        [X, Y, x, y] = props[prop].bbox
        new_mask[X:x, Y:y] = fill_contours(props[prop].image_filled.astype(np.uint8)*255)

    return new_mask

In [ ]:
threshold_prob = 0.4

for batch_idx, (data, target, fname, original_size) in enumerate(dataloaders['test']):
    # wsi image number
    wsi_image_number = fname[0].split("_")[0]
    if wsi_image_number not in wsi_tissue_patches:
        # extract the tissue region from original image and draw the heat grid
        wsi_image_path = "{}/{}.png".format(wsi_images_dir_tumor, wsi_image_number)
        wsi_image = open_wsi(wsi_image_path)
        pil_scaled_down_image = scale_down_wsi(wsi_image, magnification, False)
        np_scaled_down_image = pil_to_np(pil_scaled_down_image)
        # extract tissue region 
        np_tissue_mask, np_masked_image = extract_normal_region_from_wsi(wsi_image_path, np_scaled_down_image, None)
        pil_masked_image = np_to_pil(np_masked_image)
        # draw the heat grid
        pil_img_result, heat_grid, number_of_tiles = draw_heat_grid(np_masked_image, tile_size)
        tissue_patches = []
        for idx, (position, row, column, location, size, color) in enumerate(heat_grid):
            if color != GREEN_COLOR: 
                tissue_patches.append("{}_r{}c{}.png".format(wsi_image_number, row, column))
        wsi_tissue_patches[wsi_image_number] = tissue_patches
        #print(wsi_tissue_patches)
    # check if the patch was excluded in preprocessing step
    patch_excludde_in_preprocessing = fname[0] not in wsi_tissue_patches[wsi_image_number]
    # load the mask image
    mask_np_img = target[0].numpy()
    # roi x non_roi classes
    wsi_class = class_name if wsi_image_path.find(class_name) > 0 else "normal"
    patch_class = "roi" if np.max(np.unique(mask_np_img)) > 0 else 'non_roi'
    # load the predicted image result
    patch_results_dir = "{}/{}/patch/{}x{}/{}".format(results_dir, wsi_class, patch_size, patch_size, fname[0])
    print("Patch results dir: " + patch_results_dir)
    unet_result_img = "{}/01-unet_result/{}".format(patch_results_dir, fname[0])
    predicted_pil_img = Image.fromarray(np.zeros(mask_np_img.shape)) if patch_excludde_in_preprocessing else load_pil_image(unet_result_img, gray=True) if os.path.isfile(unet_result_img) else Image.fromarray(np.zeros(mask_np_img.shape))
    predicted_np_img = np.copy(pil_to_np(predicted_pil_img)) 
    predicted_np_img = predicted_np_img * (1.0/255)
    predicted_np_img = basic_threshold(predicted_np_img, threshold=threshold_prob, output_type="float")
    
    # SAVE BINARY IMAGES
    bin_images_path = '../../datasets/BONE_CHANNELS/results/BONE_CHANNELS__Size-640x640_Epoch-100_Images-464_Batch-1__random_distortion/testing/bones/patch/640x640/binary_images/'
    predicted_np_img_255 = predicted_np_img * 255
    mean = get_mean_props_area(predicted_np_img_255)
    predicted_np_img_255 = remove_noise(predicted_np_img_255, mean)
    predicted_np_img_255 = fill_components_contours(predicted_np_img_255)
    
    new_p = Image.fromarray(predicted_np_img_255)
    if new_p.mode != 'RGB':
        new_p = new_p.convert('RGB')
    new_p.save(bin_images_path + fname[0])




# Quantitative metrics for image-patches (512x512)

In [ ]:
import os
import sys
import csv

from scipy import ndimage as nd
from skimage import measure



current_path = os.path.abspath('.')
root_path = os.path.dirname(os.path.dirname(current_path))
sys.path.append(root_path)

from sourcecode.ORCA.orca_dataloader_512x512 import *
from sourcecode.wsi_image_utils import *
from sourcecode.evaluation_utils import *



dataset_dir = "../../datasets/ORCA_512x512"
dataset_dir_results = "/media/dalifreire/DADOS/PhD/github/tumor_regions_segmentation/datasets/ORCA_512x512"

batch_size = 1
patch_size = (512, 512)
color_model = "LAB"
dataloaders = create_dataloader(batch_size=batch_size, 
                                shuffle=False,
                                dataset_dir=dataset_dir,
                                color_model=color_model)

dataset_train_size = len(dataloaders['train'].dataset)
dataset_test_size = len(dataloaders['test'].dataset)
print("-")

tile_size = 20
magnification=0.625

threshold_prob = 0.50
threshold_itc = 200/(0.243 * pow(2, 5))

wsi_images_dir_normal = "{}/testing/normal/wsi".format(dataset_dir)
wsi_images_dir_tumor = "{}/testing/tumor/wsi".format(dataset_dir)

trained_model_version = "ORCA__Size-512x512_Epoch-352_Images-80_Batch-1__one_by_epoch"
results_dir="{}/results/{}/testing".format(dataset_dir_results, trained_model_version)
csv_file_path = "{}/quantitative_analysis_{}.csv".format(results_dir, threshold_prob)

wsi_tissue_patches = {}
with open(csv_file_path, mode='w') as medidas_file:

    medidas_writer = csv.writer(medidas_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    medidas_writer.writerow(['wsi_image', 'patch_image', 'class', 'auc', 'accuracy', 'precision', 'f1/dice', 'jaccard', 'sensitivity/recall', 'specificity', 'pixels', 'tp', 'tn', 'fp', 'fn'])

    for batch_idx, (data, target, fname, original_size) in enumerate(dataloaders['test']):
        
        # load the mask image
        mask_np_img = target[0].numpy()

        # roi x non_roi classes
        wsi_class = class_name
        patch_class = "roi"                

        # load the predicted image result
        patch_results_dir = "{}/{}/patch/{}x{}/{}".format(results_dir, wsi_class, patch_size[0], patch_size[1], fname[0])
        unet_result_img = "{}/01-unet_result/{}".format(patch_results_dir, fname[0])
        predicted_pil_img = load_pil_image(unet_result_img, gray=True) if os.path.isfile(unet_result_img) else Image.fromarray(np.zeros(mask_np_img.shape))
        predicted_np_img = np.copy(pil_to_np(predicted_pil_img))
        predicted_np_img = predicted_np_img * (1.0/255)
        predicted_np_img = basic_threshold(predicted_np_img, threshold=threshold_prob, output_type="uint8")

        predicted_labels = measure.label(predicted_np_img, connectivity=2)
        predicted_np_img = np.zeros((predicted_np_img.shape[0], predicted_np_img.shape[1]))
        labels = np.unique(predicted_labels)
        properties = measure.regionprops(predicted_labels)
        for lbl in range(1, np.max(labels)):
            major_axis_length = properties[lbl-1].major_axis_length
            if major_axis_length > threshold_itc:
                predicted_np_img[predicted_labels == lbl] = 1


        # metrics
        auc = roc_auc_score(mask_np_img, predicted_np_img)
        precision = precision_score(mask_np_img, predicted_np_img)
        recall = recall_score(mask_np_img, predicted_np_img)
        accuracy = accuracy_score(mask_np_img, predicted_np_img)
        f1 = f1_score(mask_np_img, predicted_np_img)
        specificity = specificity_score(mask_np_img, predicted_np_img)
        jaccard = jaccard_score(mask_np_img, predicted_np_img)

        total_pixels, tn, fp, fn, tp = tn_fp_fn_tp(mask_np_img, predicted_np_img)

        print(fname[0])
        print("Results for {:26} ({:7} - {:8} - {:04.2f} accuracy)".format(fname[0], patch_class, "unet", accuracy))
        #print("   Precision: \t{}".format(precision))
        #print("   Recall/Sen: \t{}".format(recall))
        #print("   F1/Dice: \t{}".format(f1))
        #print("   Accuracy: \t{}".format(accuracy))
        #print("   Specificity: {}".format(specificity))
        #print("   Jaccard: \t{}".format(jaccard))
        #print("   TP = {} TN = {} FP = {} FN = {}".format(tp, tn, fp, fn))
        #print("-")

        medidas_writer.writerow([fname[0], '-', patch_class, auc, accuracy, precision, f1, jaccard, recall, specificity, total_pixels, tp, tn, fp, fn])
